In [24]:
import pandas as pd
import numpy as np
import os
import string
import re

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

import nltk
from nltk.tokenize import WordPunctTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer as cv


In [2]:
df1 = pd.read_csv("FA-KES-Dataset.csv",encoding ='unicode_escape')
df1.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [3]:
df1 = df1.drop(['unit_id', 'article_title', 'source', 'date', 'location'], axis=1)
df1.columns = ['news', 'label']
df1

,news,label
0,Wed 05 Apr 2017 Syria attack symptoms consiste...,0
1,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,0
2,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,0
3,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,0
4,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,0
...,...,...
799,28-08-2016 Turkish Bombardment Kills 20 Civili...,1
800,17-08-2016 Martyrs as Terrorists Shell Aleppos...,1
801,03-08-2016 Chemical Attack Kills Five Syrians ...,0
802,01-08-2016 5 Killed as Russian Military Choppe...,1


In [4]:
df2 = pd.read_csv("fakenews-dataset-old.csv",error_bad_lines=False, engine="python",encoding ='unicode_escape')
df2.head()

,Unnamed: 0,id,headline,written_by,news,label,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 646,Unnamed: 647,Unnamed: 648,Unnamed: 649,Unnamed: 650,Unnamed: 651,Unnamed: 652,Unnamed: 653,Unnamed: 654,Unnamed: 655,Unnamed: 656,Unnamed: 657,Unnamed: 658,Unnamed: 659,Unnamed: 660,Unnamed: 661,Unnamed: 662,Unnamed: 663,Unnamed: 664,Unnamed: 665,Unnamed: 666,Unnamed: 667,Unnamed: 668,Unnamed: 669,Unnamed: 670,Unnamed: 671,Unnamed: 672,Unnamed: 673,Unnamed: 674,Unnamed: 675,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON â In Sonny Perdueâs telling,...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10041,U.S. Must Dig Deep to Stop Argentinaâs Lione...,David Waldstein,HOUSTON â Venezuela had a plan. It was a ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,19113,Cotton to House: âDo Not Walk the Plank and ...,Pam Key,"Sunday on ABCâs âThis Week,â while discu...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. â The beleaguered Republican...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df2 = df2.iloc[:, :6]

In [10]:
df2.columns = ['number', 'id', 'headline', 'written_by', 'news', 'label']
df2.head()

,number,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON â In Sonny Perdueâs telling,...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentinaâs Lione...,David Waldstein,HOUSTON â Venezuela had a plan. It was a ...,0
2,2,19113,Cotton to House: âDo Not Walk the Plank and ...,Pam Key,"Sunday on ABCâs âThis Week,â while discu...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. â The beleaguered Republican...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1


In [11]:
df2 = df2[:550]

In [12]:
df2 = df2.drop(['number', 'id', 'headline', 'written_by'], axis = 1)
df2.head()

,news,label
0,"WASHINGTON â In Sonny Perdueâs telling,...",0
1,HOUSTON â Venezuela had a plan. It was a ...,0
2,"Sunday on ABCâs âThis Week,â while discu...",0
3,"AUGUSTA, Me. â The beleaguered Republican...",0
4,Finian Cunningham has written extensively on...,1


In [13]:
df2

,news,label
0,"WASHINGTON â In Sonny Perdueâs telling,...",0
1,HOUSTON â Venezuela had a plan. It was a ...,0
2,"Sunday on ABCâs âThis Week,â while discu...",0
3,"AUGUSTA, Me. â The beleaguered Republican...",0
4,Finian Cunningham has written extensively on...,1
...,...,...
545,Sen. Ted Cruz ( ) commended Donald Trumpâs...,0
546,Report Copyright Violation If the Christian ev...,1
547,"For reasons unknown, Canadians are unable to a...",1
548,\nA stunning Foreign Intelligence Service (SVR...,1


In [14]:
df = pd.concat([df1, df2])
df

,news,label
0,Wed 05 Apr 2017 Syria attack symptoms consiste...,0
1,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,0
2,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,0
3,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,0
4,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,0
...,...,...
545,Sen. Ted Cruz ( ) commended Donald Trumpâs...,0
546,Report Copyright Violation If the Christian ev...,1
547,"For reasons unknown, Canadians are unable to a...",1
548,\nA stunning Foreign Intelligence Service (SVR...,1


In [15]:
# Count how many hoax and valid label
df['label'].value_counts()

1                                                                                        426
0                                                                                        378
0                                                                                        274
1                                                                                        270
 she flipped through her study cards                                                       1
 Reagan                                                                                    1
 but heâs shopping for a house outside the city. âYou come from no money for food      1
Name: label, dtype: int64

**Preprocessing Dataset**

In [20]:
# Remove digits in texts

def remove_digits(text):
    digits = [str(x) for x in range(10)]
    for d in digits:
        text = text.replace(d, ' ')
    return text

In [21]:
df['news'] = df['news'].apply(lambda x: remove_digits(x))
df.head()

,news,label
0,wed apr syria attack symptoms consiste...,0
1,fri apr at homs governor says us ...,0
2,sun apr death toll from aleppo bomb at...,0
3,wed apr aleppo bomb blast kills six sy...,0
4,sun jul syria rebels dead in fighti...,0


In [16]:
# Remove Punctuation

import string

def remove_punct(text):
  removed = "".join([i for i in text if i not in string.punctuation])
  return removed

In [17]:
df['news'] = df['news'].apply(lambda x:remove_punct(x))
df.head()

,news,label
0,Wed 05 Apr 2017 Syria attack symptoms consiste...,0
1,Fri 07 Apr 2017 at 0914 Homs governor says US ...,0
2,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,0
3,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,0
4,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,0


In [18]:
# Lowercase All Words in Sentences

df['news'] = df.news.str.lower()
df

,news,label
0,wed 05 apr 2017 syria attack symptoms consiste...,0
1,fri 07 apr 2017 at 0914 homs governor says us ...,0
2,sun 16 apr 2017 death toll from aleppo bomb at...,0
3,wed 19 apr 2017 aleppo bomb blast kills six sy...,0
4,sun 10 jul 2016 29 syria rebels dead in fighti...,0
...,...,...
545,sen ted cruz commended donald trumpâs ch...,0
546,report copyright violation if the christian ev...,1
547,for reasons unknown canadians are unable to ac...,1
548,\na stunning foreign intelligence service svr ...,1


In [19]:
print(df['news'].iloc[0])

wed 05 apr 2017 syria attack symptoms consistent with nerve agent use who victims of a suspected chemical attack in syria appeared to show symptoms consistent with reaction to a nerve agent the world health organization said on wednesday some cases appear to show additional signs consistent with exposure to organophosphorus chemicals a category of chemicals that includes nerve agents who said in a statement putting the death toll at at least 70 the united states has said the deaths were caused by sarin nerve gas dropped by syrian aircraft russia has said it believes poison gas had leaked from a rebel chemical weapons depot struck by syrian bombs sarin is an organophosporus compound and a nerve agent chlorine and mustard gas which are also believed to have been used in the past in syria are not a russian defence ministry spokesman did not say what agent was used in the attack but said the rebels had used the same chemical weapons in aleppo last year the who said it was likely that some 

In [22]:
# Remove non-english words

def non_english(data):
    data = re.sub(r'\W+', ' ', data)
    data = data.lower()
    data = data.replace("[^a-zA-Z]", " ")
    word_tokens = word_tokenize(data)
    filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
    filtered_word = [w + " " for w in filtered_word]
    return "".join(filtered_word)

In [25]:
nltk.download('punkt')
df['news'] = df['news'].apply(non_english)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
df['news']

0      wed apr syria attack symptoms consistent with ...
1      fri apr at homs governor says us attack caused...
2      sun apr death toll from aleppo bomb attack at ...
3      wed apr aleppo bomb blast kills six syrian sta...
4      sun jul syria rebels dead in fighting for key ...
                             ...                        
545    sen ted cruz commended donald s choices for hi...
546    report copyright violation if the christian ev...
547    for reasons unknown canadians are unable to ac...
548    a stunning foreign intelligence service svr re...
549    hillary clinton was campaigning for her husban...
Name: news, Length: 1354, dtype: object

In [27]:
# Train Test Split

from sklearn.model_selection import train_test_split

berita = df['news'].to_numpy()
y = df['label'].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(berita, y, test_size = 0.2) # 80:20

In [28]:
# Tokenize Data (Sentence Into Words)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)
xsequences = tokenizer.texts_to_sequences(x_train)
xssequences = tokenizer.texts_to_sequences(x_test)

x_train_new = pad_sequences(xsequences)
x_test_new = pad_sequences(xssequences)


**Build Model**

In [29]:
import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(1000, 128))
# Using 1 LSTM Layer and 3 Dense Layer
model.add(tf.keras.layers.LSTM(128, dropout=0.4))
model.add(tf.keras.layers.Dense(496, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(215, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='relu'))


model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [30]:
# Callback Function

class callback(tf.keras.callbacks.Callback):
  def stop_epoch(self, epoch, logs={}):
    if(logs.get('accuracy')>0.96): # End iteration if accuracy > 96%
      print("End of Epochs")
      self.model.stop_epoch = True

callbacks = callback()

**Training Session**

In [ ]:
model.fit(x_train_new, y_train, batch_size=128, epochs=10,
          validation_data=(x_test_new, y_test),
          verbose=1, callbacks=[callbacks])

Epoch 1/10
5/5 [==============================] - 166s 29s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 2/10
5/5 [==============================] - 147s 28s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 3/10
5/5 [==============================] - 145s 28s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 4/10
5/5 [==============================] - 178s 34s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 5/10
5/5 [==============================] - 165s 30s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 6/10
5/5 [==============================] - 150s 28s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 7/10
5/5 [==============================] - 175s 34s/step - loss: nan - accuracy: 0.4929 - val_loss: nan - val_accuracy: 0.4752
Epoch 8/10
5/5 [==============================] - 144s 26s/ste

In [ ]:
model.fit(x_train_new, y_train, batch_size=128, epochs=10,
          validation_data=(x_test_new, y_test),
          verbose=1, callbacks=[callbacks])

Epoch 1/10
9/9 [==============================] - 436s 48s/step - loss: 0.2215 - accuracy: 0.9185 - val_loss: 1.3916 - val_accuracy: 0.7778
Epoch 2/10
9/9 [==============================] - 436s 49s/step - loss: 0.2181 - accuracy: 0.9338 - val_loss: 1.4749 - val_accuracy: 0.7778
Epoch 3/10
9/9 [==============================] - 548s 59s/step - loss: 0.1951 - accuracy: 0.9329 - val_loss: 1.5832 - val_accuracy: 0.7969
Epoch 4/10
9/9 [==============================] - 328s 32s/step - loss: 0.1786 - accuracy: 0.9377 - val_loss: 1.6039 - val_accuracy: 0.8008
Epoch 5/10
9/9 [==============================] - 375s 39s/step - loss: 0.1695 - accuracy: 0.9473 - val_loss: 1.4595 - val_accuracy: 0.7893
Epoch 6/10
9/9 [==============================] - 390s 39s/step - loss: 0.1492 - accuracy: 0.9597 - val_loss: 1.7741 - val_accuracy: 0.8161
Epoch 7/10
9/9 [==============================] - 418s 43s/step - loss: 0.1793 - accuracy: 0.9569 - val_loss: 1.7394 - val_accuracy: 0.8046
Epoch 8/10
9/9 [====

**Testing Session**

In [ ]:
prediction = model.predict(x_test_new)

In [ ]:
# history = model.fit(x_test_new, y_test)

7/7 [==============================] - 61s 9s/step - loss: 0.9795 - accuracy: 0.8392


In [ ]:
history = model.fit(x_test_new, y_test)

9/9 [==============================] - 213s 24s/step - loss: 1.7493 - accuracy: 0.7816


-- It can be concluded that the model has the latest accuracy on the testing data of 78% 